1\)

In [17]:
import numpy as np

def sq_matrix_rot_180(X):
    res = np.zeros(X.shape, float)
    n = X.shape[0]
    for i in range(n):
        for j in range(n):
            res[i, j] = X[n-i-1, n-j-1]
    return res

W = np.array([[0, 1], [2, 3]], float)

def Conv2D(image, W): #stride 1, no padding
    W_rot = sq_matrix_rot_180(W)
    W_dim = W.shape[0]
    res_dim = image.shape[0] - W.shape[0] + 1
    res = np.zeros([res_dim, res_dim], float)
    for i in range(res_dim):
        for j in range(res_dim):
            res[i, j] = np.sum(W_rot*image[i:W_dim, j:W_dim])
    return res

test_img = np.array([[6, 5], [3, 2]])
print(Conv2D(test_img, W))

[[31.]]


Test

In [1]:
import scipy.signal as ss

def conv2Dd_(image,W,stride,Conv):
    if (Conv):
        y = ss.convolve2d(image, W, mode='valid') ## valid padding
    else:
        y = ss.correlate2d(image, W, mode='valid') ## valid padding
    Xdim = len(image[0])//stride
    x = np.zeros([Xdim,Xdim],float)
    if stride>1: ## implement stride
        for i in range(0,Xdim):
            for j in range(0,Xdim):
                x[i,j] = y[i*stride,j*stride]
    else:
        x = y
    return(x)